Read unclean.csv <br>
Clean 'em up <br>
Store in clean.csv <br>
<br>
Requires: https://github.com/rfk/pyenchant <br>

In [41]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string, re
from nltk.tokenize.nist import NISTTokenizer
tknzr = NISTTokenizer()
import pandas
import time
#from timeout import timeout
from math import pow
import enchant

checker = enchant.Dict("en_US")
lemmatizer = WordNetLemmatizer()
RE_D = re.compile('\d')
RE_AP = re.compile("'")
RE_DO = re.compile("\.")

In [42]:
def norm(x,n=3):
	y = pow(10,n)
	return int(x*y)/pow(10,n)

p = {}
for i in string.punctuation:
    p[i]=True
    
def replace_punc(s_out):
    s1_out = []
    for i in s_out:
        try:
            if(p[i]==True):
                continue
        except:
            s1_out.append(i)
    return s1_out

57


In [78]:
#df = pd.read_csv('unclean.csv')
df = pd.read_csv('all_GitHub.csv')
#df.describe()
# 160 rating failure
# 46653 rows
# 57 unique specialties
# 6197 unique docs

r = df['review']
R = [] # cleaned review texts

In [79]:
stop_words = set(stopwords.words("english"))
to_keep = ['she','he','not','during','before','after','him','her','his','hers','herself','himself']
# keeping gendered words to be able to run analyses later
to_remove = ['also',"i'm","i'll","we've","i've","can't","it's","i'd","you're","doesn't","that's",'got','would','many','some','few']
for i in to_keep:
    stop_words.remove(i)
for i in to_remove:
    stop_words.add(i)

In [80]:
to_allow = ['xray','x-ray']

#@timeout(5)
def pre2(r1): # created June 19
    r1 = r1.lower()
    r1 = re.sub('\s+', ' ', r1)
    r_out = tknzr.tokenize(r1)
    r_out = replace_punc(r_out)
    for i in range(len(r_out)):
        if len(r_out[i]) < 2:
            r_out[i] = ''
            continue
        if r_out[i] in to_allow:
            continue
        if checker.check(r_out[i]) == False:
            r_out[i] = ''
            continue
        if r_out[i] in stop_words:
            r_out[i] = ''
            continue
        r_out[i] = lemmatizer.lemmatize(r_out[i])  

    r_out = (' ').join(r_out)
    r_out = r_out.strip()
    r_out = r_out.lower()
    r_out = re.sub('\s+', ' ', r_out)
    r_out = r_out.split(' ')
    return r_out

In [81]:
start = time.time()
timer1 = start
len_r = len(r)
for i,r1 in enumerate(r):
    if(i%100==0):
        timer2 = time.time()
        print(i,norm(timer2-timer1))
        timer1 = timer2
    if(i%1000==0 and i != 0):
        t_so_far = norm((timer2-start)/60,0)
        print(norm(i/len_r)*100,'% done in ',t_so_far,' minutes\t',norm((t_so_far*(len_r-i))/i,0),' minutes remaining...')
    r1 = r1[2:-1]
    try:
        new2 = pre2(r1)
        R.append(new2)
    except:
        print(i, N[i])
        continue
timer2 = time.time()
print(i,norm(timer2-timer1))

0 0.001
100 0.228
200 0.132
300 0.135
400 0.142
500 0.096
600 0.077
700 0.091
800 0.121
900 0.101
1000 0.113
2.1 % done in  0.0  minutes	 0.0  minutes remaining...
1100 0.098
1200 0.08
1300 0.147
1400 0.108
1500 0.144
1600 0.108
1700 0.129
1800 0.108
1900 0.082
2000 0.106
4.2 % done in  0.0  minutes	 0.0  minutes remaining...
2100 0.106
2200 0.133
2300 0.092
2400 0.087
2500 0.114
2600 0.183
2700 0.093
2800 0.095
2900 0.134
3000 0.121
6.4 % done in  0.0  minutes	 0.0  minutes remaining...
3100 0.114
3200 0.097
3300 0.12
3400 0.141
3500 0.112
3600 0.112
3700 0.137
3800 0.11
3900 0.121
4000 0.108
8.5 % done in  0.0  minutes	 0.0  minutes remaining...
4100 0.152
4200 0.105
4300 0.088
4400 0.099
4500 0.092
4600 0.085
4700 0.124
4800 0.099
4900 0.08
5000 0.1
10.7 % done in  0.0  minutes	 0.0  minutes remaining...
5100 0.119
5200 0.158
5300 0.106
5400 0.119
5500 0.11
5600 0.099
5700 0.14
5800 0.086
5900 0.103
6000 0.117
12.8 % done in  0.0  minutes	 0.0  minutes remaining...
6100 0.1
6200 0.0

In [82]:
df['review'] = R
df.to_csv('cleaned_2.csv')

In [83]:
df.sample(5)

,Unnamed: 0,doc_id,name,specialty,gender,S,P,H,K,review
45646,7732.5,7732,dr-jane-la-grange-somerset-west-wc-za,gynecologist-obgyn,f,5,5,5,5,"[fantastic, woman, she, delivered, baby, boy, ..."
9730,1652.9,1652,Dr-Deirdre+M.-Marshall-South+Miami-FL.html,cosmetic-plastic-surgeon,f,5,5,5,5,"[year, half, ago, could, not, happier, incredi..."
37172,6325.4,6325,dr-alida-andersen-toronto-on-ca,dentist,f,5,5,5,5,"[came, emergency, after, car, accident, word, ..."
11379,1940.1,1940,Dr-Michael+F.-Bishara-Riverside-CA.html,addiction-medicine,m,5,5,5,5,"[far, one, best, doctor, ever, dealt, went, hi..."
10727,1820.7,1820,Dr-RICHARD+L.-BRIDGHAM-Somers-NY.html,orthodontist,m,5,5,5,5,"[say, enough, great, thing, orthodontics, daug..."


# Ignore infrequent specialties

In [84]:
s_freq = {}
S = df['specialty']
for s in S:
    try:
        s_freq[s] += 1
    except:
        s_freq[s] = 1

In [85]:
s_freq

{'psychiatrist': 1301,
 'bariatric-weight-loss': 302,
 'naturopath': 1408,
 'chiropractor': 1922,
 'vascular-phlebologist': 356,
 'family-gp': 1935,
 'gynecologist-obgyn': 1798,
 'pulmonologist-lungs': 195,
 'orthodontist': 1537,
 'internist-geriatrician': 1269,
 'dermatologist': 1730,
 'cosmetic-plastic-surgeon': 1928,
 'periodontist-gums': 841,
 'neurosurgeon': 1063,
 'endodontist-root-canal': 743,
 'pain-mgmt-physical-rehab': 1308,
 'acupuncturist': 1119,
 'sports-medicine-physicians': 282,
 'oral-surgeon': 1028,
 'psychologist': 1479,
 'reproductive-endocrinologist': 1601,
 'urologist': 963,
 'orthopedics-sports': 1385,
 'cardiothoracic-surgeon': 167,
 'radiologist': 206,
 'cardiologist': 791,
 'dentist': 1970,
 'oncologist-hematologist': 853,
 'nephrologist-kidney': 218,
 'gastroenterologist': 1090,
 'podiatrist': 1648,
 'rheumatologist': 467,
 'pediatrician': 1253,
 'ophthalmologist': 1465,
 'urogynecologist': 70,
 'nurse-practitioner': 384,
 'optometrist': 1816,
 'midwife': 161,

In [86]:
threshold = 1000
allowed = [] # list of allowed specialties, which have frequency above threshold
for i in s_freq:
    if s_freq[i] >= threshold:
        allowed.append(i)
print(allowed)
print(len(allowed))

['psychiatrist', 'naturopath', 'chiropractor', 'family-gp', 'gynecologist-obgyn', 'orthodontist', 'internist-geriatrician', 'dermatologist', 'cosmetic-plastic-surgeon', 'neurosurgeon', 'pain-mgmt-physical-rehab', 'acupuncturist', 'oral-surgeon', 'psychologist', 'reproductive-endocrinologist', 'orthopedics-sports', 'dentist', 'gastroenterologist', 'podiatrist', 'pediatrician', 'ophthalmologist', 'optometrist', 'ear-nose-and-throat-ent', 'endocrinologist', 'neurologist', 'surgeon-general']
26


In [87]:
df1 = df[df['specialty'].str.contains('|'.join(allowed))]

In [88]:
print(len(df1.specialty.unique())) # 26
print(len(df1.name.unique())) # 4533

26
4553


# Group specialties

In [89]:
group_specialties1 = {
 'internist-geriatrician':'int',
 'gastroenterologist':'gas',
 'family-gp':'family',
 'pain-mgmt-physical-rehab':'bone',
 'reproductive-endocrinologist':'gyn',
 'orthopedics-sports':'bone',
 'dermatologist':'skin',
 #'acupuncturist':'alt',
 #'naturopath':'alt',
 'ophthalmologist':'eye',
 'psychiatrist':'psych',
 'cosmetic-plastic-surgeon':'skin',
 #'optometrist':'eye',
 #'chiropractor':'bone',
 'pediatrician':'pedia',
 'neurosurgeon':'brain',
 #'orthodontist':'dent',
 'endocrinologist':'endo',
 'neurologist':'brain',
 #'dentist':'dent',
 #'podiatrist':'bone',
 'surgeon-general':'surg',
 'ear-nose-and-throat-ent':'ent',
 'gynecologist-obgyn':'gyn',
 #'psychologist':'psych',
 #'oral-surgeon':'dent'
}

In [90]:
for i in range(len(df1)):
    entry = df1.iloc[i]
    name = entry.name
    try:
        df1.set_value(name,'specialty', group_specialties1[entry['specialty']])
    except:
        df1.set_value(name,'specialty', '-1')
    if i%1000 == 0:
        print(i)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [91]:
df1.sample(5)

,Unnamed: 0,doc_id,name,specialty,gender,S,P,H,K,review
40042,6796.2,6796,Dr-Ingrid-Tall-Brisbane-QLD.html,skin,f,5,5,5,5,"[heard, before, decided, two, treatment, clini..."
11230,1909.6,1909,dr-akash-sinha-crestview-hills-ky-us,pedia,m,5,5,5,5,"[amazing, doctor, son, diagnosed, type, diabet..."
2134,341.6,341,Dr-SARAH-CANTIN-LANGLOIS-CONCORD-MA.html,-1,f,5,5,5,5,"[est, avant, pendant, et, la, op, me, et, la]"
24149,4131.1,4131,Dr-Reza-Shahim-North+Little+Rock-AR.html,brain,m,5,5,5,5,"[patient, year, way, back, 2000, he, performed..."
38441,6529.4,6529,Dr-Sourabh-Nagpal-New+Delhi-DL.html,-1,m,5,5,5,5,"[hotel, south, crown, came, eating, fortunatel..."


In [92]:
df1.to_csv('grouped_1.csv')

# Anonymize

In [93]:
df2 = df1.drop(columns="name") # name might still be retrievable from review text

In [94]:
df2.sample(5)

,Unnamed: 0,doc_id,specialty,gender,S,P,H,K,review
39971,6783.8,6783,-1,m,5,5,5,5,"[could, not, recommend, highly, wish, he, fami..."
25624,4386.8,4386,family,m,5,5,5,5,"[la, est, et, est, et, tr, comp]"
30318,5180.3,5180,bone,m,5,5,5,5,"[staff, always, professional, nice, phone, gre..."
41606,7052.2,7052,psych,m,4,4,5,5,"[helped, tremendously, sleep, disorder, well, ..."
25381,4344.5,4344,-1,f,5,5,5,5,"[one, best, seen, us, activator, tool, adjustm..."


In [95]:
df2.columns = ['r_id','doc_id','specialty','gender','S','P','H','K','review']
df2.sample(5)

,r_id,doc_id,specialty,gender,S,P,H,K,review
29210,5003.1,5003,-1,f,5,5,5,5,"[shore, great, she, great, person, great, psyc..."
338,56.1,56,gas,f,4,3,1,1,"[terrible, experience, cold, slightly, judgmen..."
31999,5461.4,5461,gas,m,5,5,5,5,"[helpful, great, bedside, manor, knew, he, wel..."
32249,5503.5,5503,eye,f,5,5,5,5,"[muller, best, she, ophthalmologist, 10, year,..."
15125,2584.6,2584,-1,m,5,5,5,5,"[genetics, seems, complicated, data, clue, mea..."


In [101]:
df2.to_csv('processed_1.csv',index=False)

In [102]:
df3 = pd.read_csv('processed_1.csv')
df3.sample(3)

,r_id,doc_id,specialty,gender,S,P,H,K,review
17139,3658.6,3658,-1,f,5,5,5,5,"['unusually', 'she', 'great', 'child', 'help',..."
8354,1754.8,1754,skin,f,5,5,5,5,"['cannot', 'say', 'enough', 'staff', 'plastic'..."
19197,4068.5,4068,-1,m,5,5,5,5,"['always', 'thought', 'dentist', 'butt', 'his'..."


In [103]:
for i,row in df.iterrows():
    print(type(row['review']),len(row['review']),row['review'])
    break

<class 'list'> 16 ['fabulous', 'doctor', 'really', 'take', 'his', 'time', 'listens', 'concern', 'office', 'manager', 'wonderful', 'flexible', 'scheduling', 'appointment', 'always', 'friendly']
